In [1]:
from bs4 import BeautifulSoup
import pdfplumber
import requests
import pandas as pd

In [2]:
# Record format: Source, Total plastic Mass (per km river flow), Microplastics (per km river flow), Sampling Dates, Sampling Location, Sampling Method
Data = pd.DataFrame(columns = [["Source"],["Total Plastic Mass (per km river flow)"], ["Total Microplastics (per km river flow)"], ["Sampling Dates"], ["Sampling Location"], ["Sampling Method"]])

In [3]:
# Target data spread = 2013 - 2021, across all UK main waterways
# greenpeace labs technical report - pages 16 to 17 - PDF - Feb/March 2019 - All UK
with pdfplumber.open("data/GP_mps.pdf") as pdf:
    for i in range(15,17):
        page = pdf.pages[i]
        print(page.extract_text(Layout = True))

Total number
Total plastic of plastic items Number of Equivalent number per square km
microplastics
River Sampling location Date mass (mg) per / fragments
km river flow per f k l m ow r iver (<5 r m iv m er ) f p lo e w r k m Total plastics Mic (< ro 5 p m la m s ) t ics
Exe Footbridge over the Exe in central Exeter 06/02/2019 flow rate not recorded
Exe Footbridge over the Exe in central Exeter 06/02/2019 flow rate not recorded
Exe Footbridge over the Exe in central Exeter 06/02/2019 3.76 3.47 1.45 11571 4821
Thames Mid-river, approx. 1.3km above Teddington Lock, between built up areas 11/02/2019 196.06 22.78 15.88 75938 52926
Thames Pedestrian bridge in Oxford, built up, housing estate 12/03/2019 60.88 18.53 11.47 61761 38233
Thames Footbridge, close to canal, semi-urban 12/03/2019 16.57 6.16 4.26 20518 14205
Thames Road bridge in Chertsey - lots of traffic. River wide, downstream of lock 12/03/2019 21.38 3.05 1.53 10181 5090
Severn Stone pedestrian bridge at Atcham, downstream of roa

In [4]:
# england .gov Microplastics cefas Data - CSV - 2013 to 2021 - England & Wales
cefas_dat = pd.read_csv("data/cefas_mps.csv")
cefas_dat.drop(["Rep", "Station", "Latitude", "Longitude", "Sediment dried weight in g","Gravel %", "Sand %", "Silt %", "OC2", "OC63", "Nit63", "Nit2"], axis = 1, inplace = True)
cefas_dat.head()


,Year,Number of particles,Number of particles Blank_Corrected,Number of particles per kg d.w. sediment,Coastal,Region
0,2013,41,38,7600,coastal,Northern North Sea
1,2013,61,58,11600,coastal,Northern North Sea
2,2013,45,42,8400,coastal,Northern North Sea
3,2013,12,9,1800,coastal,Northern North Sea
4,2013,20,17,3400,coastal,Northern North Sea


In [6]:
# scotland .gov Microplastics in Scotlands Seas - CSV - 2013 to Present - Scotland
scots_dat = pd.read_csv("data/Scotland_mps.csv")
scots_dat.drop(["Cruise", "Season", "Field_ID", "TOW_ID", "Year", "Decimal Latitude at deployment (WGS84)", "Decimal Longitude at deployment (WGS84)", "Tow_length (m)", "vol_filtered_m3", "WIND_SPEED", "WIND_Direction", "SEA_STATE", "Ship Speed"], axis = 1, inplace = True)
# now drop columns with Retrieve, km2, LTE, GT5
scots_dat.head()


,Region,DATE_Deploy,TIME_Deploy,FLOW_METER_READING_deploy,DATE_Retrieve,TIME_Retrieve,decLat_Retrieve,decLong_Retrieve,FLOW_METER_READING_retrieve,SUM_micro_km2,...,film_LTE5mm_count,nurdles_LTE5mm_count,paint_flakes_LTE5mm_count,fibres_GT5mm_count,beads_GT5mm_count,polystyrene_GT5mm_count,fragments_GT5mm_count,film_GT5mm_count,nurdles_GT5mm_count,paint_flakes_GT5mm_count
0,Forth and Tay,06/01/2014,16:23,8212,06/01/2014,17:05,56.08633,-2.83967,24662,5715,...,0,1,0,0,0,0,0,0,0,0
1,Forth and Tay,06/01/2014,17:24,24662,06/01/2014,18:25,56.13600,-2.69017,39113,9463,...,1,1,5,0,0,0,0,0,0,0
2,Moray Firth,08/01/2014,14:01,46519,08/01/2014,15:05,57.84333,-3.73500,79946,0,...,0,0,0,0,0,0,0,0,0,0
3,Moray Firth,08/01/2014,15:40,79946,08/01/2014,16:10,57.88667,-3.84950,300,0,...,0,0,0,0,0,0,0,0,0,0
4,Moray Firth,08/01/2014,18:50,300,08/01/2014,19:50,57.94650,-3.28083,33753,1022,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Normalise all collected data - date format, all lowercase, river/location names, units 
def normal_dat(data):
    """
    This function normalises an input DataFrame, ready for input into a larger microplastics dataset.
    
    Parameters:
    data (DataFrame): Input dataset
    
    Returns:
    norm_data (DataFrame): Normalised dataset
    """